# ITS Support Chatbot

This chatbot is an educational tool that's built to answer questions related to the CSUSB's [Information Technology Services](https://www.csusb.edu/its). The chatbot was built by team 1 for [CSE 6550: Software Engineering Concepts](https://catalog.csusb.edu/coursesaz/cse/)

In this notebook, we will demonstrate how the chatbot uses retrieval augemented generation (RAG) to answer questions using the ITS website as the primary data source.

[![GitHub](https://img.shields.io/badge/GitHub-black?style=flat&logo=github&logoColor=white)](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team1)
[![Wiki](https://img.shields.io/badge/Wiki-blue?style=flat&logo=wikipedia&logoColor=white)](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team1/wiki)

## Table of Contents
1. [Setup](#1-Setup)  
    - 1.1. [Requirements and Environment](#1.1-Requirements-and-Environment)  
    - 1.2. [Import Required Libraries](#1.2-Importing-Required-Libraries)  
    - 1.3. [Set Up Environment Variables](#1.3-Set-Up-Environment-Variables)  
2. [Building the Chatbot](#2.-Building-the-Chatbot)  
    - 2.1 [Vector Store and Embeddings](#2.1-Vector-Store-and-Embeddings)
        - 2.1.1. [Create Vector Store](#2.1.1-Function-to-fetch-the-embedding-model)  
        - 2.1.2. [Fetch Embedding Model](#2.1.2-Function-to-fetch-the-embedding-model)  
    - 2.2. [Document Handling](#2.-Document-Handling)  
        - 2.2.1. [Text Cleaning](#2.1-Function-to-Clean-Text)  
        - 2.2.2. [Clean HTML Content](#2.2-Function-to-Clean-and-Extract-Text-from-HTML-Content)  
        - 2.2.3. [Load Documents from the Web](#2.3-Function-for-loading-documents-from-the-web)
    - 2.3 [Milvus Vector Store Management](#2.3.-Milvus-Vector-Store-Management)
        - 2.3.1. [Load Existing Vector Store](#2.3.1-Function-to-load-existing-vector-store-(Milvus-database))
        - 2.3.2. [Split Documents into Chunks](#2.3.2-Function-to-split-documents)   
        - 2.3.3. [Create New Vector Store](#2.3.3-Function-to-Create-New-Vector-Store-(Milvus-database))  
        - 2.3.4. [Initialize Milvus](#2.3.4-Core-function-for-initializing-Milvus)
        - 2.3.5. [Initializing Vector Store](#2.3.5-Initializing-Vector-Store)
3. [Testing the Chatbot](#3.-Testing-the-Chatbot)  
    - 3.1. [Create RAG Prompt](#3.1-Function-to-create-RAG-prompt)  
    - 3.2. [Query RAG](#3.2-Function-to-query-RAG-model)  
    - 3.3. [Retrieve RAG Response](#3.3-Get-response-from-RAG)  
4. [Conclusion](#4-Conclusion)


## 1. Setup

Environment Installation:
- Install ipykernel and virtualenv
- Activate new virtual environment

In [1]:
!python3 -m venv chatbot
!chatbot/bin/pip install ipykernel
!chatbot/bin/python -m ipykernel install --user --name=chatbot --display-name "Python (chatbot)"
!source chatbot/bin/activate
print('Virtual Environment Created!')

  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached debugpy-1.8.9-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.1 kB)
  Using cached ipython-8.30.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached jupyter_client-8.6.3-py3-none-any.whl.metadata (8.3 kB)
  Using cached jupyter_core-5.7.2-py3-none-any.whl.metadata (3.4 kB)
  Using cached matplotlib_inline-0.1.7-py3-none-any.whl.metadata (3.9 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached psutil-6.1.0-cp36-abi3-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (22 kB)
  Using cached pyzmq-26.2.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.2 kB)
  Using cached tornado-6.4.2-cp38-abi3-manylinux_2_5_x86_64.manylinux1_x86_64.manylin

### 1.1 Requirements

**Requirements**
- Python >= 3.10

In [1]:
!python3 --version

Python 3.12.3


**Switch Kernel**

Switch the Kernel in the Jupyter Notebook by
- Go to the Menu Bar
- Select Kernel
- Select Change kernel
- From the list of available kernels select `Python (chatbot)`
- If `Python (chatbot)` is not available, refresh the page

### 1.2 Importing Required Libraries

**Purpose**: Set up the environment, configure logging, suppress warnings, and import dependencies

**Input**: None

**Output**: Prints "Dependencies imported successfully."

**Processing**:
- Configures logging with INFO level and standard format.
- Disables Hugging Face tokenizers parallelism warnings.
- Installs required dependencies silently using pip.
- Suppresses non-critical warnings for clean output.
- Imports essential libraries for vector store operations and document processing.
- Confirms successful imports with a message.

**Libraries**:

- `os`: For interacting with the operating system and managing environment variables.

- `warnings`: For suppressing warning messages during runtime.

- `pymilvus.connections` and `pymilvus.utility`: For connecting to and managing the Milvus vector database.

- `create_stuff_documents_chain` from `langchain.chains.combine_documents`: For combining documents into a chain.

- `Document` from `langchain.schema`: For representing and managing document data structures.

- `ChatPromptTemplate` from `langchain_core.prompts`: For creating prompts to structure queries for the language model.

- `langchain-groq` from `langchain_groq.chat_models`: For integrating the GROQ AI language model.

- `Milvus` from `langchain_milvus`: For using the Milvus vector database with LangChain.

- `RecursiveUrlLoader` from `langchain_community.document_loaders`: For fetching and loading documents recursively from a URL.

- `BeautifulSoup` from `bs4`: For parsing HTML and extracting relevant content.

- `RecursiveCharacterTextSplitter` from `langchain_text_splitters`: For splitting large text data into smaller chunks for easier processing.

- `create_retrieval_chain` from `langchain.chains`: For building chains to retrieve and combine relevant documents.

- `HuggingFaceEmbeddings` from `langchain_huggingface`: For generating embeddings using Hugging Face models to encode text into vectors.

- `HTTPStatusError` from `httpx`: For handling HTTP status-related exceptions.


In [3]:
import os

# Suppress Hugging Face tokenizers parallelism warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Installing dependencies if not already installed, suppressing "Requirement already satisfied" warnings
!chatbot/bin/pip install -q httpx pymilvus python-dotenv --root-user-action=ignore
!chatbot/bin/pip install -q langchain langchain-core langchain-milvus langchain-groq langchain-community beautifulsoup4 langchain-text-splitters langchain-huggingface sentence-transformers --root-user-action=ignore

import warnings
warnings.filterwarnings('ignore')

from pymilvus import connections, utility

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.schema import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq.chat_models import ChatGroq
from langchain_milvus import Milvus
from langchain_community.document_loaders import RecursiveUrlLoader
from bs4 import BeautifulSoup
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain_huggingface import HuggingFaceEmbeddings

from httpx import HTTPStatusError

from dotenv import set_key, load_dotenv

print("Dependencies imported successfully.")

Dependencies imported successfully.


### 1.3 Set Up Environment Variables

**Purpose**:

To define and load the environment variables required for the RAG (Retrieval-Augmented Generation) operation, including settings for the corpus source, API key, vector store database, and embedding model.

**Input**:
Prompted for GROQ API that can be found [here](https://console.groq.com/keys)

1. The environment variable `GROQ_API_KEY` being set in the system/environment.

**Output**:
1. Four variables are defined and available for subsequent code execution:

- `CORPUS_SOURCE`: URL of the corpus to be processed [default](https://www.csusb.edu/its).
- `GROQ_API_KEY`: API key for authenticating with the MistralAI service, fetched from system environment variables.
- `MILVUS_URI`: File path for the Milvus Lite vector database (default: `'milvus/jupyter_milvus_vector.db'`).
- `MODEL_NAME`: Name of the embedding model for vectorizing documents (default: `'sentence-transformers/all-MiniLM-L12-v2'`).
2. Prints "ENV variables defined." to indicate successful setup.

**Processing**:
1. **Set the Corpus Source**:

- Assigns the URL (`https://www.csusb.edu/its`) to the variable `CORPUS_SOURCE`.
- This specifies the website from which documents will be loaded for processing.

2. **Fetch GROQ API Key**:

- Retrieves the API key for GROQ AI using `os.environ.get("GROQ_API_KEY")`.
- This ensures secure access by fetching sensitive credentials from the system's environment variables.
3. **Set the Milvus Database Path**:

- Defines the file path (`milvus/jupyter_milvus_vector.db`) for the Milvus Lite vector store via `MILVUS_URI`.
- This will be used to store and retrieve vector embeddings.
4. **Set the Embedding Model Name**:

- Specifies the name of the Hugging Face embedding model (`sentence-transformers/all-MiniLM-L12-v2`) in `MODEL_NAME`.
- This model is used for transforming text into vector embeddings for document processing.
5. **Print Status**:

- Prints a confirmation message (`"ENV variables defined."`) to confirm successful setup.



In [5]:
env_file = ".env"

# Load existing environment variables from .env if it exists
if os.path.exists(env_file):
    load_dotenv(env_file)

api_key = input("Please enter your GROQ API key: ").strip()

# Save the API key to the .env file
if "GROQ_API_KEY" not in os.environ:
    set_key(env_file, "GROQ_API_KEY", api_key)
else:
    print("No API key entered. Operation canceled.")
    
CORPUS_SOURCE = 'https://www.csusb.edu/its'
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
MILVUS_URI = "milvus/jupyter_milvus_vector.db"
MODEL_NAME = "sentence-transformers/all-MiniLM-L12-v2"

print("ENV variables defined.")

Please enter your GROQ API key: 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
No API key entered. Operation canceled.
ENV variables defined.


## 2.Building the Chatbot

### 2.1 Vector Store and Embeddings

#### 2.1.1 Function to Check Vector Store (Milvus database)

**Purpose**:
To check if the Milvus vector store already exists at the specified URI.

**Input**: Path to the Milvus database `uri` (str).

**Output**:
Returns a boolean (True if the vector store exists, False otherwise).

**Processing**:
- Creates the `/milvus` directory if it doesn’t exist.
- Connects to the Milvus database at the specified uri.
- Checks if the collection IT_support exists in the Milvus database.

In [6]:
def vector_store_check(uri):
    """
    Returns response on whether the vector storage exists

    Returns:
        boolean
    """
    # Create the directory if it does not exist
    head = os.path.split(uri)
    os.makedirs(head[0], exist_ok=True)

    # Connect to the Milvus database
    connections.connect("default", uri=uri)

    # Return True if exists, False otherwise
    return utility.has_collection("IT_support")

print("Function `vector_store_check` defined.")

Function `vector_store_check` defined.


#### 2.1.2 Function to fetch the embedding model

**Purpose**:
To load and initialize the embedding model for vectorizing documents.

**Input**:
None.

**Output**: Returns the embedding function loaded from the Hugging Face model specified in `MODEL_NAME`.

**Processing**:
- Loads the embedding model using `HuggingFaceEmbeddings`.
- Returns the initialized embedding function.

In [7]:
def get_embedding_function():
    """
    Returns embedding function for the model

    Returns:
        embedding function
    """
    embedding_function = HuggingFaceEmbeddings(model_name=MODEL_NAME)

    return embedding_function

print("Function `get_embedding_function` defined.")

Function `get_embedding_function` defined.


### 2.2. Document Handling

#### 2.2.1 Function to Clean Text
**Purpose**:
To clean a given text by removing extra whitespace and blank lines.

**Input**: The input text to be cleaned `text` (str).

**Output**:
Returns the cleaned text with unnecessary whitespace and blank lines removed.

**Processing**:
- Splits the text into lines and trims leading/trailing spaces from each line.
- Removes empty lines from the text.
- Joins the cleaned lines into a single string.

In [8]:
def clean_text(text):
    """Further clean the text by removing extra whitespace and new lines."""
    lines = (line.strip() for line in text.splitlines())
    cleaned_lines = [line for line in lines if line]
    return '\n'.join(cleaned_lines)

print("Function `clean_text` defined.")

Function `clean_text` defined.


#### 2.2.2 Function to Clean and Extract Text from HTML Content
**Purpose**:
To extract and clean the main content from an HTML document.

**Input**: The HTML content to be cleaned `html_content` (str).

**Output**:
Returns the cleaned plain text content extracted from the HTML.

**Processing**:

- Parses the HTML using `BeautifulSoup`.
- Removes unnecessary elements like `<script>, <style>, <header>, <footer>, and <nav>`.
- Extracts text from the `<main>` tag if it exists, or the entire document otherwise.
- Cleans the extracted text using the clean_text function.

In [9]:
def clean_text_from_html(html_content):
    """Clean HTML content to extract main text."""
    soup = BeautifulSoup(html_content, 'html.parser')

    # Remove unnecessary elements
    for script_or_style in soup(['script', 'style', 'header', 'footer', 'nav']):
        script_or_style.decompose()

    main_content = soup.find('main')
    if main_content:
        content = main_content.get_text(separator='\n')
    else:
        content = soup.get_text(separator='\n')

    return clean_text(content)

print("Function `clean_text_from_html` defined.")

Function `clean_text_from_html` defined.


#### 2.2.3 Function for loading documents from the web

**Purpose**:
To recursively load and clean documents from a web source specified in `CORPUS_SOURCE`.

**Input**:
None.

**Output**:
Returns a list of cleaned documents as `Document` objects.

**Processing**:

- Uses RecursiveUrlLoader to load all pages from the base URL (`CORPUS_SOURCE`).
- Iterates through the loaded documents:
- Cleans the text using `clean_text_from_html`.
- Creates `Document` objects with the cleaned text and metadata.
- Returns the list of cleaned `Document` objects.

In [10]:
def load_documents_from_web():
    """
    Load the documents from the web and store the page contents

    Returns:
        list: The documents loaded from the web
    """
    loader = RecursiveUrlLoader(
        url=CORPUS_SOURCE,
        prevent_outside=True,
        base_url=CORPUS_SOURCE
        )
    raw_documents = loader.load()

    # Ensure documents are cleaned
    cleaned_documents = []
    for doc in raw_documents:
        cleaned_text = clean_text_from_html(doc.page_content)
        cleaned_documents.append(Document(page_content=cleaned_text, metadata=doc.metadata))

    return cleaned_documents

print("Function `load_documents_from_web` defined.")

Function `load_documents_from_web` defined.


### 2.3 Milvus Vector Store Management

#### 2.3.1 Function to load existing vector store (Milvus database)

**Purpose**: To connect to an existing Milvus vector store.  

**Input**: Path to the Milvus database (`uri`).  

**Output**: Returns the loaded vector store as a Milvus object.  

**Processing**:
- Connects to the specified Milvus database.
- Initializes a Milvus object with the collection IT_support and the embedding function from `get_embedding_function`.

In [11]:
def load_existing_db(uri=MILVUS_URI):
    """
    Load an existing vector store from the local Milvus database specified by the URI.

    Args:
        uri (str, optional): Path to the local milvus db. Defaults to MILVUS_URI.

    Returns:
        vector_store: The vector store created
    """
    # Load an existing vector store
    vector_store = Milvus(
        collection_name="IT_support",
        embedding_function=get_embedding_function(),
        connection_args={"uri": uri},
    )

    print("Vector store loaded")
    return vector_store

print("Function `load_existing_db` defined.")

Function `load_existing_db` defined.


#### 2.3.2 Function to split documents

**Purpose**: To split large documents into smaller chunks for better processing and context preservation.  

**Input**: List of documents to be split.  

**Output**: Returns a list of document chunks.  

**Processing**:

- Creates a `RecursiveCharacterTextSplitter` with a chunk size of 1000 characters and an overlap of 300 characters.
- Splits each document into smaller chunks using the text splitter.
- Returns the list of document chunks.

In [12]:
def split_documents(documents):
    """
    Split the documents into chunks

    Args:
        documents (list): The documents to split

    Returns:
        list: list of chunks of documents
    """
    # Create a text splitter to split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=300,
        is_separator_regex=False,
    )

    # Split the documents into chunks
    docs = text_splitter.split_documents(documents)

    print("Documents successfully split")
    return docs

print("Function `split_documents` defined.")

Function `split_documents` defined.


#### 2.3.3 Function to Create New Vector Store (Milvus database)

**Purpose**:
To create a new vector store in Milvus using the given documents and embedding function.

**Input(s)**:

- `docs` (list): Documents to store in the vector database.
- `embeddings`: The embedding function for vectorizing documents.
- `uri` (str): Path to the Milvus database.  

**Output**: Returns the newly created vector store.  

**Processing**:

- Connects to Milvus at the specified uri.
- Creates a new collection named IT_support, dropping any existing one.
- Embeds the documents and stores them in the vector database.

In [13]:
def create_vector_store(docs, embeddings, uri):
    """
    This function initializes a vector store using the provided documents and embeddings.

    Args:
        docs (list): A list of documents to be stored in the vector store.
        embeddings : A function or model that generates embeddings for the documents.
        uri (str): Path to the local milvus db

    Returns:
        vector_store: The vector store created
    """
    # Create a new vector store and drop any existing one
    vector_store = Milvus.from_documents(
        documents=docs,
        embedding=embeddings,
        collection_name="IT_support",
        connection_args={"uri": uri},
        drop_old=True,
    )

    print("Vector store created")
    return vector_store

print("Function `create_vector_store` defined.")

Function `create_vector_store` defined.


#### 2.3.4 Core function for initializing Milvus

**Purpose**: To initialize the Milvus vector store by either loading an existing one or creating a new one.

**Input**: Path to the Milvus database `uri`.  

**Output**: Returns the initialized vector store.  

**Processing**:
- Checks if the vector store already exists using vector_store_check.
- If it exists:  
    - loads it using `load_existing_db`.  
- Otherwise:
    - Loads documents from the web using `load_documents_from_web`.
    - Splits the documents into chunks using `split_documents`.
    - Creates a new vector store using `create_vector_store`.

In [14]:
def initialize_milvus(uri: str=MILVUS_URI):
    """
    Initialize the vector store for the RAG model

    Args:
        uri (str, optional): Path to the local vector storage. Defaults to MILVUS_URI.

    Returns:
        vector_store: The vector store created
    """
    if vector_store_check(uri):
        vector_store = load_existing_db(uri)
        print("Embeddings loaded from existing storage")
    else:
        embeddings = get_embedding_function()
        print("Embeddings Loaded")
        documents = load_documents_from_web()
        print("Documents Loaded")

        # Split the documents into chunks
        docs = split_documents(documents=documents)
        print("Documents Splitting completed")

        vector_store = create_vector_store(docs, embeddings, uri)
    print("Milvus successfully initialized")
    return vector_store

print("Function `initialize_milvus` defined.")

Function `initialize_milvus` defined.


#### 2.3.5 Initializing vector store (Milvus database)
**Purpose**: To initialize the Milvus database system for vector storage and retrieval.  

**Input**: None.  

**Output**: None.  

**Processing**: Calls the `initialize_milvus` function to set up the Milvus vector database system.

In [15]:
print("Starting Milvus initialization.")
initialize_milvus()

Starting Milvus initialization.


ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

## 3. Testing the Chatbot

### 3.1 Function to create RAG prompt

**Purpose**:
To create a prompt template for the RAG model with predefined system instructions

**Input**: None

**Output**: Returns a `ChatPromptTemplate` object

**Processing**:
- Defines a template with system instructions for generating accurate and context-based responses
- Initializes a `ChatPromptTemplate` using the system template and a human prompt structure

In [ ]:
def create_prompt():
    """
    Create a prompt template for the RAG model

    Returns:
        PromptTemplate: The prompt template for the RAG model
    """
    # Define the prompt template
    PROMPT_TEMPLATE = """\
    You are an AI assistant that provides answers strictly based on the provided context. Adhere to these guidelines:
     - Only answer questions based on the content within the <context> tags.
     - If the <context> does not contain information related to the question, respond only with: "I don't have enough information to answer this question."
     - For unclear questions or questions that lack specific context, request clarification from the user.
     - Provide specific, concise ansewrs. Where relevant information includes statistics or numbers, include them in the response.
     - Avoid adding any information, assumption, or external knowledge. Answer accurately within the scope of the given context and do not guess.
     - If information is missing, respond only with: "I don't have enough information to answer this question."
    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", PROMPT_TEMPLATE),
        ("human", "<question>{input}</question>\n\n<context>{context}</context>"),
    ])

    print("Prompt template defined")
    return prompt

print("Function `create_prompt` defined.")

### 3.2 Function to query RAG model

**Purpose**: To query the RAG model for a response to a user’s question.

**Input**: User’s question query.

**Output**: Returns the generated response from the RAG model, including source references.

**Processing**:

- Initializes the MistralAI model.
- Loads the prompt template using `create_prompt`.
- Loads the vector store and creates a retriever to fetch relevant documents.
- Creates a document chain and a retrieval chain for generating responses.
- Executes the query and generates a response using the retrieval chain.
- Extracts and appends source metadata to the response.

In [ ]:
def query_rag(query):
    """
    Entry point for the RAG model to generate an answer to a given query

    This function initializes the RAG model, sets up the necessary components such as the prompt template, vector store,
    retriever, document chain, and retrieval chain, and then generates a response to the provided query.

    Args:
        query (str): The query string for which an answer is to be generated.

    Returns:
        str: The answer to the query
    """
    # Define the model
    model = ChatGroq(model='llama-3.1-70b-versatile', temperature = 0)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
    print("Model Loaded")

    prompt = create_prompt()

    # Load the vector store and create the retriever
    vector_store = load_existing_db(uri=MILVUS_URI)
    retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"score_threshold": 0.7, "k":5})
    try:
        document_chain = create_stuff_documents_chain(model, prompt)
        print("Document Chain Created")

        retrieval_chain = create_retrieval_chain(retriever, document_chain)
        print("Retrieval Chain Created")

        # Generate a response to the query
        response = retrieval_chain.invoke({"input": f"{query}"})
    except HTTPStatusError as e:
        print(f"HTTPStatusError: {e}")
        if e.response.status_code == 429:
            error_message = "I am currently experiencing high traffic. Please try again later."
            print(error_message)
            return error_message, []
        error_message = "I am unable to answer this question at the moment. Please try again later."
        print(error_message)
        return error_message, []

    # logic to add sources to the response
    max_relevant_sources = 4 # number of sources at most to be added to the response
    all_sources = ""
    sources = []
    count = 1
    for i in range(max_relevant_sources):
        try:
            source = response["context"][i].metadata["source"]
            # check if the source is already added to the list
            if source not in sources:
                sources.append(source)
                all_sources += f"[Source {count}]({source}), "
                count += 1
        except IndexError: # if there are no more sources to add
            break
    all_sources = all_sources[:-2] # remove the last comma and space
    response["answer"] += f"\n\nSources: {all_sources}"
    print("------------------------------------------------------------------------")
    print("Response Generated:\n")

    return response["answer"]

print("Function `query_rag` defined.")

### 3.3 Get response from RAG

**Purpose**: Taking input query from user, executing it and displaying the fetched output.  

**Input**: User's question.

**Output**: Response generated from the `query_rag` function.

**Processing**:
- User input: Prompts the user to input a query.
- Query execution: Passes the user-provided query to the `query_rag` function.
- Response display: Prints the response generated.

In [ ]:
response = query_rag(input("Enter your query: "))

print(response)

## 4. Conclusion

 **Recap**:

- Developed an **ITS Support chatbot** using the **RAG (Retrieval-Augmented Generation)** system, which retrieves relevant documents and generates responses based on the context provided.
- Integrated **widgets** in **Jupyter Notebook** for interactive user input and response display, enabling real-time query processing.
- Configured the chatbot with **natural language processing models** like **GROQ AI** and **vector-based document retrieval** using **Milvus**, ensuring accurate and context-aware responses.

**Next Steps**:

- Expand the chatbot's knowledge base to accommodate a broader range of questions or integrate multiple datasets from diverse sources, enhancing the system’s robustness and versatility.
- Improve the chatbot’s performance and extend its capabilities by incorporating more advanced features, such as deeper integrations with external knowledge sources and enhanced response-generation techniques.

**Resources**:

- "The **ITS Support Chatbot** project was developed by **CSUSB Fall 2024 CSE6550 Team 1**  to provide automated assistance for IT-related queries and technical support."

[![GitHub](https://img.shields.io/badge/GitHub-black?style=flat&logo=github&logoColor=white)](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team1)
[![Wiki](https://img.shields.io/badge/Wiki-blue?style=flat&logo=wikipedia&logoColor=white)](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team1/wiki)
